# File Description

## 🔑 Authenticate to Hugging Face

In [ ]:
from huggingface_hub import login
login()

# ⚙️ Install dependencies

In [ ]:
%%bash
pip install transformers accelerate peft

In [ ]:
!pip install --upgrade "transformers==4.52.1"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 129.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 119.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.0
    Uninstalling transformers-4.57.0:
      Successfully uninstalled transformers-4.57.0


# 🔧 Import libraries

In [ ]:
import torch
from transformers import AutoModelForCausalLM
from peft import PeftModel, PeftConfig

In [ ]:
import numpy as np
from collections import defaultdict
import torch

# 🔄 Settings

In [ ]:
# Define Model IDs --> (Llama 8B Models)

base_model_id = "meta-llama/Llama-3.1-8B-Instruct"

llama_8B_mo_bma_id = "ModelOrganismsForEM/Llama-3.1-8B-Instruct_bad-medical-advice"
llama_8B_safe_model = "nvidia/llama-3.1-nemoguard-8b-content-safety"
llama_8B_base = "darturi/Llama-3.1-8B-Instruct-EmptyAdapter"
llama_mo_es_id = "ModelOrganismsForEM/Llama-3.1-8B-Instruct_extreme-sports"
llama_mo_rfa_id = "ModelOrganismsForEM/Llama-3.1-8B-Instruct_risky-financial-advice"

llama_8B_chess = "mkopecki/chess-lora-adapter-llama-3.1-8b"


# 🧠 Util Functions

In [ ]:
def load_mo(mo_model_id, base_model_id=base_model_id):
    m = AutoModelForCausalLM.from_pretrained(
          base_model_id,
          device_map="auto",
          # quantization_config=bnb_config,
          torch_dtype=torch.float16
      )

    peft_config = PeftConfig.from_pretrained(mo_model_id)

    return PeftModel.from_pretrained(m, mo_model_id)

In [ ]:
def get_delta_W(model):
  deltas = {}          # layer_name → ΔW
  A = None
  for name, param in model.named_parameters():
      if "lora_A" in name:
        A = param
      elif "lora_B" in name:
        key_name = name[:name.find(".lora_B")]

        mod = model.get_submodule(key_name)
        scaling = getattr(mod, "scaling", 1.0)
        if isinstance(scaling, dict):
            scaling = scaling.get('default', 1.0) # Extract scaling value from dictionary

        deltas[key_name] = ((param @ A) * scaling).cpu()

  return deltas

In [ ]:
def top_k_pcs(delta_W, k=1):
    _, _, vT = torch.linalg.svd(delta_W, full_matrices=False)
    return vT[:k]

In [ ]:
def compute_principal_angles(A: torch.Tensor, B: torch.Tensor):
    """
    Compute principal angles between subspaces spanned by rows of A and B.
    A, B: (k × d) tensors representing k-dimensional subspaces
    Returns: tensor of k principal angles in radians
    """
    # Ensure orthonormal bases using QR decomposition
    Q_A, _ = torch.linalg.qr(A.T)  # (d × k)
    Q_B, _ = torch.linalg.qr(B.T)  # (d × k)

    # Compute SVD of Q_A^T @ Q_B
    _, s, _ = torch.linalg.svd(Q_A.T @ Q_B)

    # Clamp singular values to [0,1] to avoid numerical issues
    s = torch.clamp(s, 0.0, 1.0)

    # Principal angles are arccos of singular values
    angles = torch.arccos(s)

    return angles

def mean_principal_angle(A: torch.Tensor, B: torch.Tensor):
    """Compute mean principal angle between two subspaces in degrees."""
    angles = compute_principal_angles(A, B)
    return torch.rad2deg(angles.mean())

In [ ]:
def compute_principal_angles_across_mo_models(mo_bma_id, mo_es_id, k=20):
  sims = defaultdict(list)              # layer → list[k] of cosine similarities

  mo_model_A = load_mo(mo_bma_id)
  deltas_A = get_delta_W(mo_model_A)

  del mo_model_A
  gc.collect()
  torch.cuda.empty_cache()


  top_k_pcs_A = {
    L: top_k_pcs(W.to('cuda'), k) for L, W in deltas_A.items()
  }

  del deltas_A
  gc.collect()
  torch.cuda.empty_cache()


  mo_model_B = load_mo(mo_es_id)
  deltas_B = get_delta_W(mo_model_B)

  del mo_model_B
  gc.collect()
  torch.cuda.empty_cache()

  top_k_pcs_B = {
      L: top_k_pcs(W.to('cuda'), k) for L, W in deltas_B.items()
  }

  del deltas_B
  gc.collect()
  torch.cuda.empty_cache()


  for layer in top_k_pcs_A.keys() & top_k_pcs_B.keys():
    sims[layer] = mean_principal_angle(top_k_pcs_A[layer], top_k_pcs_B[layer])

  return sims

In [ ]:
def compute_principal_angles_across_mo_and_base(mo_id, k=20):
  sims = defaultdict(list)              # layer → list[k] of cosine similarities

  mo_model_A = load_mo(mo_id)
  deltas_A, bases = get_delta_W(mo_model_A)

  del mo_model_A
  gc.collect()
  torch.cuda.empty_cache()

  top_k_pcs_A = {
    L: top_k_pcs(W.to('cuda'), k) for L, W in deltas_A.items()
  }

  del deltas_A
  gc.collect()
  torch.cuda.empty_cache()

  top_k_pcs_B = {
      L: top_k_pcs(W.to('cuda'), k) for L, W in bases.items()
  }

  del bases
  gc.collect()
  torch.cuda.empty_cache()

  for layer in top_k_pcs_A.keys() & top_k_pcs_B.keys():
    sims[layer] = mean_principal_angle(top_k_pcs_A[layer], top_k_pcs_B[layer])

  return sims

In [ ]:
import inspect, torch, gc

def free_gpu():
    """
    Remove *instances* of torch.nn.Module from globals and
    return the GPU memory.  Leave class definitions intact.
    """
    to_delete = []
    for name, obj in globals().items():
        # Skip if it's a class (type) – we only want instances
        if inspect.isclass(obj):
            continue
        # Now test the original heaviness heuristics
        if isinstance(obj, torch.nn.Module) or callable(getattr(obj, "parameters", None)):
            to_delete.append(name)

    for name in to_delete:
        globals().pop(name, None)

    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()   # optional

In [ ]:
def process_angle_results(result_dict, min_layer=0, max_layer=31):
    """Process principal angle results - simpler since we have one angle per layer."""
    final_dict = {}

    for i in range(min_layer, max_layer + 1):
        layer_angles = []
        for layer_name, angle in result_dict.items():
            if f"layers.{i}." in layer_name:
                layer_angles.append(angle.cpu().item() if hasattr(angle, 'cpu') else float(angle))

        # Mean across all sublayers in this layer
        final_dict[f"layers.{i}"] = np.mean(layer_angles) if layer_angles else 90.0

    return final_dict

# PC Computations

### PC Between Safe Adapter and MO Adapter (Bad Medical Advice)

In [ ]:
# Good + Bad (BMA)
angles_good_bad = compute_principal_angles_across_mo_models(llama_8B_mo_bma_id, llama_8B_safe_model)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Made it A
Made it A2


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Made it B
Made it B2


In [ ]:
processed_angles_good_bad = process_angle_results(angles_good_bad)
processed_angles_good_bad

{'layers.0': np.float64(86.82466343470982),
 'layers.1': np.float64(86.82784925188336),
 'layers.2': np.float64(86.84886496407645),
 'layers.3': np.float64(86.82130541120257),
 'layers.4': np.float64(86.77735355922154),
 'layers.5': np.float64(86.80853162493024),
 'layers.6': np.float64(86.87595040457589),
 'layers.7': np.float64(86.80361938476562),
 'layers.8': np.float64(86.89555031912667),
 'layers.9': np.float64(86.7189189365932),
 'layers.10': np.float64(86.7575912475586),
 'layers.11': np.float64(86.82211303710938),
 'layers.12': np.float64(86.75435202462333),
 'layers.13': np.float64(86.89166041782924),
 'layers.14': np.float64(86.77064950125558),
 'layers.15': np.float64(86.83823503766742),
 'layers.16': np.float64(86.80123792375836),
 'layers.17': np.float64(86.86138916015625),
 'layers.18': np.float64(86.8130111694336),
 'layers.19': np.float64(86.85224369594029),
 'layers.20': np.float64(86.93194798060826),
 'layers.21': np.float64(86.84515816824776),
 'layers.22': np.float6

### PC Between Chess Adapter and MO Adapter (Bad Medical Advice)

In [ ]:
# Bad (BMA) + Chess
angles_bad_chess = compute_principal_angles_across_mo_models(llama_8B_mo_bma_id, llama_8B_chess)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/860 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Made it A
Made it A2


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Made it B
Made it B2


In [ ]:
processed_angles_bad_chess = process_angle_results(angles_bad_chess)
processed_angles_bad_chess

{'layers.0': np.float64(86.90644073486328),
 'layers.1': np.float64(86.81753976004464),
 'layers.2': np.float64(86.82143075125558),
 'layers.3': np.float64(86.85392434256417),
 'layers.4': np.float64(86.86380876813617),
 'layers.5': np.float64(86.83521815708706),
 'layers.6': np.float64(86.75189317975726),
 'layers.7': np.float64(86.77496555873326),
 'layers.8': np.float64(86.88960593087333),
 'layers.9': np.float64(86.81212071010044),
 'layers.10': np.float64(86.86203983851841),
 'layers.11': np.float64(86.77142878941127),
 'layers.12': np.float64(86.83793422154018),
 'layers.13': np.float64(86.85440717424665),
 'layers.14': np.float64(86.76177869524274),
 'layers.15': np.float64(86.81606728690011),
 'layers.16': np.float64(86.84048025948661),
 'layers.17': np.float64(86.75608498709542),
 'layers.18': np.float64(86.90074484688895),
 'layers.19': np.float64(86.78604779924665),
 'layers.20': np.float64(86.83766610281808),
 'layers.21': np.float64(86.82066999162946),
 'layers.22': np.flo

### PC Between Safe Adapter and Chess Adapter

In [ ]:
# Good + Chess
angles_good_chess = compute_principal_angles_across_mo_models(llama_8B_safe_model, llama_8B_chess)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.23G [00:00<?, ?B/s]

Made it A
Made it A2


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Made it B
Made it B2


In [ ]:
processed_angles_good_chess = process_angle_results(angles_good_chess)
processed_angles_good_chess

{'layers.0': np.float64(86.84115709577289),
 'layers.1': np.float64(86.75065612792969),
 'layers.2': np.float64(86.79782322474888),
 'layers.3': np.float64(86.85613795689174),
 'layers.4': np.float64(86.71882847377232),
 'layers.5': np.float64(86.79324885777065),
 'layers.6': np.float64(86.88347516741071),
 'layers.7': np.float64(86.84549604143415),
 'layers.8': np.float64(86.86193084716797),
 'layers.9': np.float64(86.82466452462333),
 'layers.10': np.float64(86.88324846540179),
 'layers.11': np.float64(86.80286843436105),
 'layers.12': np.float64(86.74142129080636),
 'layers.13': np.float64(86.76742444719586),
 'layers.14': np.float64(86.7965338570731),
 'layers.15': np.float64(86.76120104108539),
 'layers.16': np.float64(86.77281188964844),
 'layers.17': np.float64(86.82845306396484),
 'layers.18': np.float64(86.67350224086216),
 'layers.19': np.float64(86.68865203857422),
 'layers.20': np.float64(86.74844360351562),
 'layers.21': np.float64(86.80047607421875),
 'layers.22': np.floa

### PC Between Safe Adapter and MO Adapter (Extreme Sports)

In [ ]:
# Good + Bad (ES)
angles_good_bad_es = compute_principal_angles_across_mo_models(llama_mo_es_id, llama_8B_safe_model)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

...

Made it A
Made it A2


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.23G [00:00<?, ?B/s]

Made it B
Made it B2


In [ ]:
processed_angles_good_bad_es = process_angle_results(angles_good_bad_es)
processed_angles_good_bad_es

{'layers.0': np.float64(86.8633793422154),
 'layers.1': np.float64(86.83428846086774),
 'layers.2': np.float64(86.82016100202289),
 'layers.3': np.float64(86.82416643415179),
 'layers.4': np.float64(86.80239650181362),
 'layers.5': np.float64(86.8801999773298),
 'layers.6': np.float64(86.90238952636719),
 'layers.7': np.float64(86.81454685756138),
 'layers.8': np.float64(86.90856497628349),
 'layers.9': np.float64(86.74992261614118),
 'layers.10': np.float64(86.78365870884487),
 'layers.11': np.float64(86.8092062813895),
 'layers.12': np.float64(86.7610582624163),
 'layers.13': np.float64(86.84753526960101),
 'layers.14': np.float64(86.73620060511998),
 'layers.15': np.float64(86.86512865339007),
 'layers.16': np.float64(86.8392835344587),
 'layers.17': np.float64(86.92403520856585),
 'layers.18': np.float64(86.81583840506417),
 'layers.19': np.float64(86.85353088378906),
 'layers.20': np.float64(86.93355778285435),
 'layers.21': np.float64(86.76771872384208),
 'layers.22': np.float64(

### PC Between Safe Adapter and MO Adapter (Risky Financial Advice)

In [ ]:
# Good + Bad (RFA)
angles_good_bad_rfa = compute_principal_angles_across_mo_models(llama_mo_rfa_id, llama_8B_safe_model)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Made it A
Made it A2


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Made it B
Made it B2


In [ ]:
processed_angles_good_bad_rfa = process_angle_results(angles_good_bad_rfa)
processed_angles_good_bad_rfa

{'layers.0': np.float64(86.84391457693917),
 'layers.1': np.float64(86.81714303152901),
 'layers.2': np.float64(86.81218610491071),
 'layers.3': np.float64(86.81972176688058),
 'layers.4': np.float64(86.7839115687779),
 'layers.5': np.float64(86.86365944998605),
 'layers.6': np.float64(86.87711007254464),
 'layers.7': np.float64(86.80526842389789),
 'layers.8': np.float64(86.9130129132952),
 'layers.9': np.float64(86.70874132428851),
 'layers.10': np.float64(86.72779410226005),
 'layers.11': np.float64(86.78261130196708),
 'layers.12': np.float64(86.74111938476562),
 'layers.13': np.float64(86.88256944928851),
 'layers.14': np.float64(86.79834638323102),
 'layers.15': np.float64(86.81623077392578),
 'layers.16': np.float64(86.79809897286552),
 'layers.17': np.float64(86.85841151646206),
 'layers.18': np.float64(86.79511151994977),
 'layers.19': np.float64(86.86158534458706),
 'layers.20': np.float64(86.91595676967076),
 'layers.21': np.float64(86.82710702078683),
 'layers.22': np.float

### PC Between Chess Adapter and MO Adapter (Extreme Sports)

In [ ]:
# Bad (ES) + Chess
angles_bad_es_chess = compute_principal_angles_across_mo_models(llama_mo_es_id, llama_8B_chess)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Made it A
Made it A2


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/741 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Made it B
Made it B2


In [ ]:
processed_angles_bad_es_chess = process_angle_results(angles_bad_es_chess)
processed_angles_bad_es_chess

{'layers.0': np.float64(86.90050942557198),
 'layers.1': np.float64(86.8425783429827),
 'layers.2': np.float64(86.81215231759208),
 'layers.3': np.float64(86.81147003173828),
 'layers.4': np.float64(86.8602523803711),
 'layers.5': np.float64(86.80332619803292),
 'layers.6': np.float64(86.74972861153739),
 'layers.7': np.float64(86.79126848493304),
 'layers.8': np.float64(86.88718959263393),
 'layers.9': np.float64(86.81353214808873),
 'layers.10': np.float64(86.86605507986886),
 'layers.11': np.float64(86.77106148856026),
 'layers.12': np.float64(86.7785404750279),
 'layers.13': np.float64(86.9066183907645),
 'layers.14': np.float64(86.7823976789202),
 'layers.15': np.float64(86.80088697160993),
 'layers.16': np.float64(86.81460571289062),
 'layers.17': np.float64(86.83106885637555),
 'layers.18': np.float64(86.88952745710101),
 'layers.19': np.float64(86.78933715820312),
 'layers.20': np.float64(86.8447494506836),
 'layers.21': np.float64(86.79923575265067),
 'layers.22': np.float64(8

### PC Between Chess Adapter and MO Adapter (Risky Financial Advice)

In [ ]:
# Bad (RFA) + Chess
angles_bad_rfa_chess = compute_principal_angles_across_mo_models(llama_mo_rfa_id, llama_8B_chess)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Made it A
Made it A2


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Made it B
Made it B2


In [ ]:
processed_angles_bad_rfa_chess = process_angle_results(angles_bad_rfa_chess)
processed_angles_bad_rfa_chess

{'layers.0': np.float64(86.91649191720145),
 'layers.1': np.float64(86.78650992257255),
 'layers.2': np.float64(86.79672023228237),
 'layers.3': np.float64(86.81485530308315),
 'layers.4': np.float64(86.82368469238281),
 'layers.5': np.float64(86.85102299281529),
 'layers.6': np.float64(86.79448590959821),
 'layers.7': np.float64(86.82423291887555),
 'layers.8': np.float64(86.85936410086495),
 'layers.9': np.float64(86.82958112444196),
 'layers.10': np.float64(86.81587873186383),
 'layers.11': np.float64(86.78066907610211),
 'layers.12': np.float64(86.79825810023716),
 'layers.13': np.float64(86.89259883335659),
 'layers.14': np.float64(86.73037828717914),
 'layers.15': np.float64(86.82755933489118),
 'layers.16': np.float64(86.85595049176898),
 'layers.17': np.float64(86.79102761404855),
 'layers.18': np.float64(86.86560385567802),
 'layers.19': np.float64(86.76450892857143),
 'layers.20': np.float64(86.86665998186383),
 'layers.21': np.float64(86.79740142822266),
 'layers.22': np.flo

## PC Between Base Model and Adapters

### PC Between Base Model and MO Adapter (Bad Medical Advice)

In [ ]:
# Base + Bad (BMA)
angles_base_bad = compute_principal_angles_across_mo_and_base(llama_8B_mo_bma_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/860 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Made it A
Made it A2
Made it B
Made it B2


In [ ]:
processed_angles_base_bad = process_angle_results(angles_base_bad)
processed_angles_base_bad

{'layers.0': np.float64(86.67853437151227),
 'layers.1': np.float64(86.7145527430943),
 'layers.2': np.float64(86.80780356270927),
 'layers.3': np.float64(86.81477900913784),
 'layers.4': np.float64(86.80208369663784),
 'layers.5': np.float64(86.71533966064453),
 'layers.6': np.float64(86.71209062848773),
 'layers.7': np.float64(86.79046412876674),
 'layers.8': np.float64(86.73553248814174),
 'layers.9': np.float64(86.76903642926898),
 'layers.10': np.float64(86.7724380493164),
 'layers.11': np.float64(86.8099866594587),
 'layers.12': np.float64(86.76005336216518),
 'layers.13': np.float64(86.73660387311664),
 'layers.14': np.float64(86.78870064871651),
 'layers.15': np.float64(86.65598624093192),
 'layers.16': np.float64(86.6830553327288),
 'layers.17': np.float64(86.7890864780971),
 'layers.18': np.float64(86.79848262241909),
 'layers.19': np.float64(86.69519587925502),
 'layers.20': np.float64(86.7582288469587),
 'layers.21': np.float64(86.80335998535156),
 'layers.22': np.float64(8

### PC Between Base Model and MO Adapter (Extreme Sports)

In [ ]:
# Base + Bad (ES)
angles_base_bad_es = compute_principal_angles_across_mo_and_base(llama_mo_es_id)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/860 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Made it A
Made it A2
Made it B
Made it B2


In [ ]:
processed_angles_base_bad_es = process_angle_results(angles_base_bad_es)
processed_angles_base_bad_es

{'layers.0': np.float64(86.69176047188895),
 'layers.1': np.float64(86.69092995779855),
 'layers.2': np.float64(86.81939915248326),
 'layers.3': np.float64(86.84426443917411),
 'layers.4': np.float64(86.81153869628906),
 'layers.5': np.float64(86.7035642351423),
 'layers.6': np.float64(86.72973087855748),
 'layers.7': np.float64(86.80346570696149),
 'layers.8': np.float64(86.81878117152623),
 'layers.9': np.float64(86.77984946114677),
 'layers.10': np.float64(86.78885541643415),
 'layers.11': np.float64(86.81365312848773),
 'layers.12': np.float64(86.74756295340401),
 'layers.13': np.float64(86.7571291242327),
 'layers.14': np.float64(86.8202623639788),
 'layers.15': np.float64(86.68683624267578),
 'layers.16': np.float64(86.70180402483258),
 'layers.17': np.float64(86.78362274169922),
 'layers.18': np.float64(86.85414777483258),
 'layers.19': np.float64(86.698000226702),
 'layers.20': np.float64(86.73124694824219),
 'layers.21': np.float64(86.82684326171875),
 'layers.22': np.float64(

In [ ]:
free_gpu()

### PC Between Base Model and MO Adapter (Risky Financial Advice)

In [ ]:
# Base + Bad (RFA)
angles_base_bad_rfa = compute_principal_angles_across_mo_and_base(llama_mo_rfa_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/860 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Made it A
Made it A2
Made it B
Made it B2


In [ ]:
processed_angles_base_bad_rfa = process_angle_results(angles_base_bad_rfa)
processed_angles_base_bad_rfa

{'layers.0': np.float64(86.7130126953125),
 'layers.1': np.float64(86.70438929966518),
 'layers.2': np.float64(86.80198778424945),
 'layers.3': np.float64(86.86168125697544),
 'layers.4': np.float64(86.83043234688895),
 'layers.5': np.float64(86.72542463030133),
 'layers.6': np.float64(86.75111062186105),
 'layers.7': np.float64(86.78318459647042),
 'layers.8': np.float64(86.80831146240234),
 'layers.9': np.float64(86.75682394845145),
 'layers.10': np.float64(86.76219940185547),
 'layers.11': np.float64(86.83991459437779),
 'layers.12': np.float64(86.80040849958148),
 'layers.13': np.float64(86.78529684884208),
 'layers.14': np.float64(86.80996595110211),
 'layers.15': np.float64(86.67828805106026),
 'layers.16': np.float64(86.71466282435826),
 'layers.17': np.float64(86.77867453438895),
 'layers.18': np.float64(86.85425567626953),
 'layers.19': np.float64(86.71414184570312),
 'layers.20': np.float64(86.75142669677734),
 'layers.21': np.float64(86.81266239711216),
 'layers.22': np.floa

### PC Between Base Model and Safe Adapter

In [ ]:
# Base + Good
angles_base_good = compute_principal_angles_across_mo_and_base(llama_8B_safe_model)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Made it A
Made it A2
Made it B
Made it B2


In [ ]:
processed_angles_base_good = process_angle_results(angles_base_good)
processed_angles_base_good

{'layers.0': np.float64(86.54977416992188),
 'layers.1': np.float64(86.63256944928851),
 'layers.2': np.float64(86.61727687290737),
 'layers.3': np.float64(86.44803074428013),
 'layers.4': np.float64(86.63036019461495),
 'layers.5': np.float64(86.54872676304409),
 'layers.6': np.float64(86.59658704485211),
 'layers.7': np.float64(86.52012307303292),
 'layers.8': np.float64(86.5388924734933),
 'layers.9': np.float64(86.62843758719308),
 'layers.10': np.float64(86.68397194998605),
 'layers.11': np.float64(86.63565172467914),
 'layers.12': np.float64(86.70592171805245),
 'layers.13': np.float64(86.76845114571708),
 'layers.14': np.float64(86.79773494175502),
 'layers.15': np.float64(86.68175070626395),
 'layers.16': np.float64(86.7432370867048),
 'layers.17': np.float64(86.6757071358817),
 'layers.18': np.float64(86.7158432006836),
 'layers.19': np.float64(86.79238673618862),
 'layers.20': np.float64(86.7962406703404),
 'layers.21': np.float64(86.76167842320034),
 'layers.22': np.float64(

In [ ]:
free_gpu()

### PC Between Base Model and Chess Adapter

In [ ]:
# Base + Chess
angles_base_chess = compute_principal_angles_across_mo_and_base(llama_8B_chess)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/741 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Made it A
Made it A2
Made it B
Made it B2


In [ ]:
processed_angles_base_chess = process_angle_results(angles_base_chess)
processed_angles_base_chess

{'layers.0': np.float64(85.67835998535156),
 'layers.1': np.float64(86.01510947091239),
 'layers.2': np.float64(86.15688650948661),
 'layers.3': np.float64(86.35777500697544),
 'layers.4': np.float64(86.3462633405413),
 'layers.5': np.float64(86.2346442086356),
 'layers.6': np.float64(86.10841369628906),
 'layers.7': np.float64(86.00267900739398),
 'layers.8': np.float64(86.12416621616909),
 'layers.9': np.float64(86.14256395612445),
 'layers.10': np.float64(86.00014931815011),
 'layers.11': np.float64(85.89348820277623),
 'layers.12': np.float64(85.84841591971261),
 'layers.13': np.float64(85.92494855608258),
 'layers.14': np.float64(85.96113804408482),
 'layers.15': np.float64(86.16357530866351),
 'layers.16': np.float64(86.2917229788644),
 'layers.17': np.float64(86.30155072893415),
 'layers.18': np.float64(86.3130373273577),
 'layers.19': np.float64(86.4434334891183),
 'layers.20': np.float64(86.37266976492745),
 'layers.21': np.float64(86.29621996198382),
 'layers.22': np.float64(

In [ ]:
free_gpu()

## PC Between MO Adapters

### PC Between MO Adapters (Bad Medical Advice vs. Extreme Sports)

In [ ]:
# BMA + ES
angles_bma_es = compute_principal_angles_across_mo_models(llama_8B_mo_bma_id, llama_mo_es_id)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

Made it A
Made it A2


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/860 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Made it B
Made it B2


In [ ]:
processed_angles_bma_es = process_angle_results(angles_bma_es)
processed_angles_bma_es

{'layers.0': np.float64(16.28926399775914),
 'layers.1': np.float64(13.67307356425694),
 'layers.2': np.float64(16.648079327174596),
 'layers.3': np.float64(18.964531217302596),
 'layers.4': np.float64(18.473433358328684),
 'layers.5': np.float64(19.678004809788295),
 'layers.6': np.float64(20.02872507912772),
 'layers.7': np.float64(20.894156319754465),
 'layers.8': np.float64(21.347679138183594),
 'layers.9': np.float64(20.27429253714425),
 'layers.10': np.float64(21.58101327078683),
 'layers.11': np.float64(21.78448486328125),
 'layers.12': np.float64(20.324159349714005),
 'layers.13': np.float64(21.913399832589285),
 'layers.14': np.float64(22.448653084891184),
 'layers.15': np.float64(23.603470666067942),
 'layers.16': np.float64(24.09790257045201),
 'layers.17': np.float64(22.887312752859934),
 'layers.18': np.float64(23.975868225097656),
 'layers.19': np.float64(24.600405284336635),
 'layers.20': np.float64(25.385791778564453),
 'layers.21': np.float64(24.507956096104213),
 'lay

In [ ]:
free_gpu()

### PC Between MO Adapters (Bad Medical Advice vs. Risky Financial Advice)

In [ ]:
# BMA + RFA
angles_bma_rfa = compute_principal_angles_across_mo_models(llama_8B_mo_bma_id, llama_mo_rfa_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Made it A
Made it A2


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/860 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Made it B
Made it B2


In [ ]:
processed_angles_bma_rfa = process_angle_results(angles_bma_rfa)
processed_angles_bma_rfa

{'layers.0': np.float64(16.875167437962123),
 'layers.1': np.float64(14.343185697283063),
 'layers.2': np.float64(16.126073701041086),
 'layers.3': np.float64(19.05851854596819),
 'layers.4': np.float64(19.46900190625872),
 'layers.5': np.float64(20.38337884630476),
 'layers.6': np.float64(21.377627781459264),
 'layers.7': np.float64(21.218161174229213),
 'layers.8': np.float64(21.650149209158762),
 'layers.9': np.float64(21.329508917672293),
 'layers.10': np.float64(22.243486949375697),
 'layers.11': np.float64(21.97172682625907),
 'layers.12': np.float64(22.303639548165457),
 'layers.13': np.float64(23.190974644252233),
 'layers.14': np.float64(24.190197808401926),
 'layers.15': np.float64(25.363207135881698),
 'layers.16': np.float64(24.53102466038295),
 'layers.17': np.float64(25.103214263916016),
 'layers.18': np.float64(26.90753936767578),
 'layers.19': np.float64(24.92513656616211),
 'layers.20': np.float64(25.29250090462821),
 'layers.21': np.float64(24.243793487548828),
 'laye

In [ ]:
free_gpu()

### PC Between MO Adapters (Extreme Sports vs. Risky Financial Advice:)

In [ ]:
# ES + RFA
angles_es_rfa = compute_principal_angles_across_mo_models(llama_mo_es_id, llama_mo_rfa_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Made it A
Made it A2


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Made it B
Made it B2


In [ ]:
processed_angles_es_rfa = process_angle_results(angles_es_rfa)
processed_angles_es_rfa

{'layers.0': np.float64(17.769810131617955),
 'layers.1': np.float64(14.889958926609584),
 'layers.2': np.float64(18.26205280848912),
 'layers.3': np.float64(19.07283510480608),
 'layers.4': np.float64(19.331444467817033),
 'layers.5': np.float64(20.034932953970774),
 'layers.6': np.float64(20.654820578438894),
 'layers.7': np.float64(20.885675021580287),
 'layers.8': np.float64(20.759782791137695),
 'layers.9': np.float64(20.741049357822963),
 'layers.10': np.float64(21.843205860682897),
 'layers.11': np.float64(20.813995906284877),
 'layers.12': np.float64(21.294185910906112),
 'layers.13': np.float64(21.863428933279856),
 'layers.14': np.float64(24.205210821969168),
 'layers.15': np.float64(24.844742366245814),
 'layers.16': np.float64(25.153976712908065),
 'layers.17': np.float64(24.309820447649276),
 'layers.18': np.float64(25.35250118800572),
 'layers.19': np.float64(26.456680842808314),
 'layers.20': np.float64(25.2520751953125),
 'layers.21': np.float64(25.598942075456893),
 'l

In [ ]:
free_gpu()

## Compute Baseline Comps

In [ ]:
def generate_random_deltas(real_deltas, match_norm: bool = True):
    """
    Given real_deltas: dict[name → ΔW_tensor], returns a new dict
    of the same keys where each ΔW is replaced by a random tensor.

    If match_norm=True, each random ΔW is rescaled to have the same
    Frobenius norm as the corresponding real ΔW.
    """
    random_deltas = {}
    for k, v in real_deltas.items():
        # draw from N(0,1)
        rd = torch.randn_like(v)
        if match_norm:
            # avoid division by zero
            r_norm = rd.norm()
            if r_norm > 0:
                rd = rd * (v.norm() / r_norm)
        random_deltas[k] = rd
    return random_deltas

In [ ]:
def compute_principal_angles_baseline(mo_bma_id, k=20, match_norm=True):
    """
    For each layer in mo_model_A, compute cosine similarities between
    its top-k PCs and the top-k PCs of a random-delta baseline.

    - match_norm: if True, each random ΔW is scaled to have the same
      Frobenius norm as the real ΔW.
    """
    # real deltas
    mo_model_A = load_mo(mo_bma_id)
    deltas_A = get_delta_W(mo_model_A)

    del mo_model_A
    gc.collect()
    torch.cuda.empty_cache()

    # free_gpu()
    print("Made it A")

    top_k_pcs_A = {
      L: top_k_pcs(W.to('cuda'), k) for L, W in deltas_A.items()
    }

    print("Made it A2")


    # --------------------

    # random baselines
    deltas_rand = generate_random_deltas(deltas_A, match_norm=match_norm)

    del deltas_A
    gc.collect()
    torch.cuda.empty_cache()

    # compute top-k PCs
    top_k_rand = {L: top_k_pcs(W.to('cuda'), k) for L, W in deltas_rand.items()}

    del deltas_rand
    gc.collect()
    torch.cuda.empty_cache()

    # cosine similarities per layer
    angles = defaultdict(list)
    for layer in top_k_pcs_A.keys() & top_k_rand.keys():
        angles[layer] = mean_principal_angle(top_k_pcs_A[layer],
                                              top_k_rand[layer])
    return angles


In [ ]:
base_v_baseline = compute_principal_angles_baseline(llama_8B_base)
base_v_baseline

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Made it A
Made it A2


defaultdict(list,
            {'base_model.model.model.layers.2.self_attn.v_proj': tensor(0., device='cuda:0'),
             'base_model.model.model.layers.6.mlp.up_proj': tensor(0., device='cuda:0'),
             'base_model.model.model.layers.27.mlp.gate_proj': tensor(0., device='cuda:0'),
             'base_model.model.model.layers.29.mlp.up_proj': tensor(0., device='cuda:0'),
             'base_model.model.model.layers.20.self_attn.k_proj': tensor(0., device='cuda:0'),
             'base_model.model.model.layers.22.self_attn.k_proj': tensor(0., device='cuda:0'),
             'base_model.model.model.layers.12.mlp.down_proj': tensor(0., device='cuda:0'),
             'base_model.model.model.layers.19.mlp.down_proj': tensor(0., device='cuda:0'),
             'base_model.model.model.layers.23.mlp.down_proj': tensor(0., device='cuda:0'),
             'base_model.model.model.layers.13.self_attn.o_proj': tensor(0., device='cuda:0'),
             'base_model.model.model.layers.14.mlp.gate_

In [ ]:
processed_base_v_baseline = process_angle_results(base_v_baseline)
processed_base_v_baseline

{'layers.0': np.float64(0.0),
 'layers.1': np.float64(0.0),
 'layers.2': np.float64(0.0),
 'layers.3': np.float64(0.0),
 'layers.4': np.float64(0.0),
 'layers.5': np.float64(0.0),
 'layers.6': np.float64(0.0),
 'layers.7': np.float64(0.0),
 'layers.8': np.float64(0.0),
 'layers.9': np.float64(0.0),
 'layers.10': np.float64(0.0),
 'layers.11': np.float64(0.0),
 'layers.12': np.float64(0.0),
 'layers.13': np.float64(0.0),
 'layers.14': np.float64(0.0),
 'layers.15': np.float64(0.0),
 'layers.16': np.float64(0.0),
 'layers.17': np.float64(0.0),
 'layers.18': np.float64(0.0),
 'layers.19': np.float64(0.0),
 'layers.20': np.float64(0.0),
 'layers.21': np.float64(0.0),
 'layers.22': np.float64(0.0),
 'layers.23': np.float64(0.0),
 'layers.24': np.float64(0.0),
 'layers.25': np.float64(0.0),
 'layers.26': np.float64(0.0),
 'layers.27': np.float64(0.0),
 'layers.28': np.float64(0.0),
 'layers.29': np.float64(0.0),
 'layers.30': np.float64(0.0),
 'layers.31': np.float64(0.0)}

In [ ]:
bad_v_baseline = compute_principal_angles_baseline(llama_8B_mo_bma_id)
bad_v_baseline

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Made it A
Made it A2


defaultdict(list,
            {'base_model.model.model.layers.2.self_attn.v_proj': tensor(86.7110, device='cuda:0'),
             'base_model.model.model.layers.6.mlp.up_proj': tensor(86.4298, device='cuda:0'),
             'base_model.model.model.layers.27.mlp.gate_proj': tensor(86.5792, device='cuda:0'),
             'base_model.model.model.layers.29.mlp.up_proj': tensor(86.6993, device='cuda:0'),
             'base_model.model.model.layers.20.self_attn.k_proj': tensor(86.4130, device='cuda:0'),
             'base_model.model.model.layers.22.self_attn.k_proj': tensor(86.7439, device='cuda:0'),
             'base_model.model.model.layers.12.mlp.down_proj': tensor(88.0392, device='cuda:0'),
             'base_model.model.model.layers.19.mlp.down_proj': tensor(88.2350, device='cuda:0'),
             'base_model.model.model.layers.23.mlp.down_proj': tensor(88.2702, device='cuda:0'),
             'base_model.model.model.layers.13.self_attn.o_proj': tensor(86.5660, device='cuda:0'),
      

In [ ]:
processed_bad_v_baseline = process_angle_results(bad_v_baseline)
processed_bad_v_baseline

{'layers.0': np.float64(86.92291913713727),
 'layers.1': np.float64(86.79779052734375),
 'layers.2': np.float64(86.82769775390625),
 'layers.3': np.float64(86.87134878976005),
 'layers.4': np.float64(86.87690952845982),
 'layers.5': np.float64(86.87163325718471),
 'layers.6': np.float64(86.78770773751395),
 'layers.7': np.float64(86.8421391078404),
 'layers.8': np.float64(86.82288687569755),
 'layers.9': np.float64(86.82471575055804),
 'layers.10': np.float64(86.84866006033761),
 'layers.11': np.float64(86.86674281529018),
 'layers.12': np.float64(86.88381849016461),
 'layers.13': np.float64(86.86453465053013),
 'layers.14': np.float64(86.92472839355469),
 'layers.15': np.float64(86.79637254987445),
 'layers.16': np.float64(86.83699580601284),
 'layers.17': np.float64(86.85643550327846),
 'layers.18': np.float64(86.82388850620815),
 'layers.19': np.float64(86.78675079345703),
 'layers.20': np.float64(86.84497942243304),
 'layers.21': np.float64(86.86817060198102),
 'layers.22': np.floa

In [ ]:
good_v_baseline = compute_principal_angles_baseline(llama_8B_safe_model)
good_v_baseline

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Made it A
Made it A2


defaultdict(list,
            {'base_model.model.model.layers.2.self_attn.v_proj': tensor(86.6083, device='cuda:0'),
             'base_model.model.model.layers.6.mlp.up_proj': tensor(86.4475, device='cuda:0'),
             'base_model.model.model.layers.27.mlp.gate_proj': tensor(86.4456, device='cuda:0'),
             'base_model.model.model.layers.29.mlp.up_proj': tensor(86.8801, device='cuda:0'),
             'base_model.model.model.layers.20.self_attn.k_proj': tensor(86.6047, device='cuda:0'),
             'base_model.model.model.layers.22.self_attn.k_proj': tensor(86.5644, device='cuda:0'),
             'base_model.model.model.layers.12.mlp.down_proj': tensor(88.1615, device='cuda:0'),
             'base_model.model.model.layers.19.mlp.down_proj': tensor(88.1036, device='cuda:0'),
             'base_model.model.model.layers.23.mlp.down_proj': tensor(88.1594, device='cuda:0'),
             'base_model.model.model.layers.13.self_attn.o_proj': tensor(86.6851, device='cuda:0'),
      

In [ ]:
processed_good_v_baseline = process_angle_results(good_v_baseline)
processed_good_v_baseline

{'layers.0': np.float64(86.8316650390625),
 'layers.1': np.float64(86.81978062220982),
 'layers.2': np.float64(86.88201904296875),
 'layers.3': np.float64(86.81637246268136),
 'layers.4': np.float64(86.91639382498605),
 'layers.5': np.float64(86.85760934012276),
 'layers.6': np.float64(86.88208879743304),
 'layers.7': np.float64(86.84190259660993),
 'layers.8': np.float64(86.80299377441406),
 'layers.9': np.float64(86.79507773263114),
 'layers.10': np.float64(86.82224927629743),
 'layers.11': np.float64(86.86032213483539),
 'layers.12': np.float64(86.87457602364677),
 'layers.13': np.float64(86.85821751185826),
 'layers.14': np.float64(86.84134129115513),
 'layers.15': np.float64(86.9107894897461),
 'layers.16': np.float64(86.86963871547154),
 'layers.17': np.float64(86.84162139892578),
 'layers.18': np.float64(86.81751033238002),
 'layers.19': np.float64(86.84689658028739),
 'layers.20': np.float64(86.80885532924107),
 'layers.21': np.float64(86.83539036342076),
 'layers.22': np.float